In [10]:
from random import random, seed
from itertools import product
import numpy as np
from icecream import ic

In [11]:
UNIVERSE_SIZE=1000
NUM_SETS=100
DENSITY=0.2

rng= np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000*DENSITY)]))


In [12]:
SETS=np.random.random((NUM_SETS, UNIVERSE_SIZE)) <DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s]= True

COSTS=np.pow(SETS.sum(axis=1), 1.1)
COSTS

array([317.37567255, 308.10319271, 343.46817808, 332.2627678 ,
       356.58323011, 311.80918295, 345.33901163, 349.08344012,
       330.39850421, 381.05553287, 308.10319271, 330.39850421,
       347.21076699, 345.33901163, 350.95702696, 369.74228963,
       315.5191819 , 352.83152354, 377.28101248, 358.4604323 ,
       347.21076699, 326.67284883, 352.83152354, 300.70339686,
       311.80918295, 392.39939734, 321.09161243, 345.33901163,
       343.46817808, 377.28101248, 330.39850421, 350.95702696,
       341.59827047, 308.10319271, 347.21076699, 364.09738836,
       287.79370589, 326.67284883, 315.5191819 , 350.95702696,
       345.33901163, 343.46817808, 339.72929293, 297.00967256,
       302.55181014, 347.21076699, 352.83152354, 339.72929293,
       332.2627678 , 365.97814428, 356.58323011, 364.09738836,
       364.09738836, 364.09738836, 343.46817808, 330.39850421,
       350.95702696, 360.3385286 , 347.21076699, 302.55181014,
       321.09161243, 356.58323011, 326.67284883, 332.26

In [13]:
def valid(solution):
    return np.all(np.logical_or.reduce(SETS[solution]))

def cost(solution):
    return COSTS[solution].sum()

In [14]:
#dumb solution of "all" sets
solution=np.full(NUM_SETS, True)
ic(valid(solution), cost(solution))
None


ic| valid(solution): np.True_
    cost(solution): np.float64(34153.846198748346)


In [15]:
#a random solution with random 50% of the sets
solution=rng.random(NUM_SETS)<0.5
valid(solution), cost(solution)
None

# Simple RHMC

In [16]:
def multiple_mutation(solution: np.ndarray, strength: float=0.3)-> np.ndarray:
    new_sol=solution.copy()
    mask=rng.random(NUM_SETS)<strength
    if not np.any(mask):
        mask[np.random.randint(NUM_SETS)]=True
    new_sol=np.logical_xor(solution, mask)
    return new_sol

In [17]:
def fitness(solution: np.ndarray):
    return (valid(solution), -cost(solution))

In [18]:
solution=rng.random(NUM_SETS)<0.3
solution_fitness=fitness(solution)
ic(fitness(solution))


tweak=multiple_mutation

strength=0.5
for steps in range(100):
    if steps%1_000:
        strength/=2
    new_solution=multiple_mutation(solution, strength)
    if fitness(new_solution)>solution_fitness:
        solution=new_solution
        solution_fitness=fitness(solution)

ic(fitness(solution))

ic| fitness(solution): (np.False_, np.float64(-10425.267492715635))
ic| fitness(solution): (np.True_, np.float64(-7458.970748713975))


(np.True_, np.float64(-7458.970748713975))

# Greedy Algorithm

In [19]:
def set_cover_greedy(SETS, COSTS):
    universe = set(range(SETS.shape[1])) #all elements
    covered = set()                      #covered elements
    selected = []                        #indexes of selected subsets, we start from an empty solution

    while covered!=universe:
        best_subset = None
        best_ratio = float("inf")

        for i in range(len(SETS)):
            subset = SETS[i]                            
            subset_elements = set(np.where(subset)[0]) #for each subset
            new_elements = subset_elements - covered   #I take the uncovered elements

            if new_elements:                  #if there are uncovered elements  
                current_cost = COSTS[i]       #I compute the ration cost/coverage
                coverage = len(new_elements)
                ratio = current_cost/coverage

                if ratio < best_ratio:        #if the ratio is the best to this moment
                    best_ratio = ratio        #I choose the current as best subset
                    best_subset = i

        if best_subset is not None:
            selected.append(best_subset)
            covered.update(set(np.where(SETS[best_subset])[0]))
        else:
            raise RuntimeError("No valid subset found. The problem may not be solvable with these sets and costs.")

    return selected

In [20]:
solution = set_cover_greedy(SETS=SETS, COSTS=COSTS)
ic(fitness(solution))

ic| fitness(solution): (np.True_, np.float64(-5600.820242414958))


(np.True_, np.float64(-5600.820242414958))